In [1]:
# Standard libraries
import pandas as pd
import json
import numpy as np
import pandas as pd
from scipy import stats
from copy import copy
import pyprind

# Plots and visualizations
import matplotlib.pyplot as plt
import seaborn as sns

df_raw = pd.read_json("stocks.json", lines=True)
df_raw.head(2)

    

,_id,Company,Price,Earnings Date,description,20-Day Simple Moving Average,200-Day Simple Moving Average,50-Day,52-Week,Analyst Recom,Average True Range,Average Volume,Beta,Change,EPS ttm,ROI,ratio,performance
0,{'$oid': '52853800bb1177ca391c17ff'},Agilent Technologies Inc.,50.44,{'$date': '2013-11-14T21:30:00.000+0000'},"{'Country': 'USA', 'Sector': 'Healthcare', 'In...",-0.0172,0.1062,"[0.0728, -0.054400000000000004]","[0.4378, -0.054400000000000004]",1.6,0.86,2569.36,1.50,-0.0148,2.68,0.163,"{'quick': 2.3, 'current': 3}","{'Year': 0.4242, 'Half Year': 0.1439, 'Month':..."
1,{'$oid': '52853800bb1177ca391c1800'},"Alcoa, Inc.",9.02,{'$date': '2013-10-08T20:30:00.000+0000'},"{'Country': 'USA', 'Sector': 'Basic Materials'...",-0.0192,0.0823,"[0.1579, -0.0925]","[0.1899, -0.0925]",3.1,0.30,26728.11,2.02,0.0033,0.25,0.007,"{'quick': 0.7000000000000001, 'current': 1.2}","{'Year': 0.09630000000000001, 'Half Year': 0.0..."


In [2]:
# Utility function

def resumetable(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    #summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values

    return summary

resumetable(df_raw)

Dataset Shape: (6756, 18)


,Name,dtypes,Missing,First Value,Second Value,Third Value
0,_id,object,0,{'$oid': '52853800bb1177ca391c17ff'},{'$oid': '52853800bb1177ca391c1800'},{'$oid': '52853800bb1177ca391c1801'}
1,Company,object,0,Agilent Technologies Inc.,"Alcoa, Inc.",WCM/BNY Mellon Focused Growth ADR ETF
2,Price,float64,13,50.44,9.02,36.4
3,Earnings Date,object,2198,{'$date': '2013-11-14T21:30:00.000+0000'},{'$date': '2013-10-08T20:30:00.000+0000'},None
4,description,object,0,"{'Country': 'USA', 'Sector': 'Healthcare', 'In...","{'Country': 'USA', 'Sector': 'Basic Materials'...","{'Country': 'USA', 'Sector': 'Financial', 'Ind..."
5,20-Day Simple Moving Average,float64,14,-0.0172,-0.0192,-0.0054
6,200-Day Simple Moving Average,float64,14,0.1062,0.0823,0.0693
7,50-Day,object,0,"[0.0728, -0.054400000000000004]","[0.1579, -0.0925]","[0.0792, -0.0194]"
8,52-Week,object,0,"[0.4378, -0.054400000000000004]","[0.1899, -0.0925]","[0.2727, -0.0194]"
9,Analyst Recom,float64,2591,1.6,3.1,NaN


In [3]:
for col in df_raw:
    print(col)
    print(type(df_raw[col].values[0]))
    print("---------------")

_id
<class 'dict'>
---------------
Company
<class 'str'>
---------------
Price
<class 'numpy.float64'>
---------------
Earnings Date
<class 'dict'>
---------------
description
<class 'dict'>
---------------
20-Day Simple Moving Average
<class 'numpy.float64'>
---------------
200-Day Simple Moving Average
<class 'numpy.float64'>
---------------
50-Day
<class 'list'>
---------------
52-Week
<class 'list'>
---------------
Analyst Recom
<class 'numpy.float64'>
---------------
Average True Range
<class 'numpy.float64'>
---------------
Average Volume
<class 'numpy.float64'>
---------------
Beta
<class 'numpy.float64'>
---------------
Change
<class 'numpy.float64'>
---------------
EPS ttm
<class 'numpy.float64'>
---------------
ROI
<class 'numpy.float64'>
---------------
ratio
<class 'dict'>
---------------
performance
<class 'dict'>
---------------


In [4]:
_id_column = df_raw["_id"].apply(lambda x: x.get('$oid'))
# np.datetime64
_id_column

0       52853800bb1177ca391c17ff
1       52853800bb1177ca391c1800
2       52853800bb1177ca391c1801
3       52853800bb1177ca391c1802
4       52853800bb1177ca391c1803
                  ...           
6751    52853810bb1177ca391c325e
6752    52853810bb1177ca391c325f
6753    52853810bb1177ca391c3260
6754    52853810bb1177ca391c3261
6755    52853810bb1177ca391c3262
Name: _id, Length: 6756, dtype: object

In [5]:
date_column = df_raw["Earnings Date"].apply(lambda x: np.nan if x is None else x.get('$date'))
date_column = pd.to_datetime(date_column)
date_column

0      2013-11-14 21:30:00+00:00
1      2013-10-08 20:30:00+00:00
2                            NaT
3                            NaT
4                            NaT
                  ...           
6751                         NaT
6752                         NaT
6753   2013-11-05 13:30:00+00:00
6754   2013-12-05 21:30:00+00:00
6755   2013-11-20 13:30:00+00:00
Name: Earnings Date, Length: 6756, dtype: datetime64[ns, UTC]

In [6]:
desc_column = df_raw["description"].apply(
    lambda x: x if not None else {"Country": None, "Sector": None, "Industry": None})
desc_column = desc_column.fillna("null")

In [7]:
perf_column = df_raw["performance"].apply(
    lambda x: x if not None else {"Year": None, "Half Year": None, "Month": None, "Week": None})
perf_column = perf_column.apply(lambda x: {"HalfYear" if k == "Half Year"  else k:v for k,v in x.items()}
                                   )
perf_column[0]

{'Year': 0.4242,
 'HalfYear': 0.1439,
 'Month': 0.0063,
 'Week': 0.0031000000000000003}

In [9]:
ratio_column = df_raw["ratio"].apply(lambda x: x if not None else {"quick": None, "current": None})
ratio_column = ratio_column.fillna("null")

In [10]:
df_clean = df_raw
df_clean["_id"] = _id_column
df_clean["Earnings Date"] = date_column
df_clean["description"] = desc_column
df_clean["performance"] = perf_column
df_clean["ratio"] = ratio_column
df_clean.columns = ["id", "company", "price", "earningsdate", 
"description", "SMA20", "SMA200", "days50highlow", 
"weeks52highlow", "analyst_recommendation", 
"avg_true_rate", "avg_vol", "beta", "change", 
"EPSttm", "ROI", "ratio", "performance"]

df_clean.head(2)


,id,company,price,earningsdate,description,SMA20,SMA200,days50highlow,weeks52highlow,analyst_recommendation,avg_true_rate,avg_vol,beta,change,EPSttm,ROI,ratio,performance
0,52853800bb1177ca391c17ff,Agilent Technologies Inc.,50.44,2013-11-14 21:30:00+00:00,"{'Country': 'USA', 'Sector': 'Healthcare', 'In...",-0.0172,0.1062,"[0.0728, -0.054400000000000004]","[0.4378, -0.054400000000000004]",1.6,0.86,2569.36,1.50,-0.0148,2.68,0.163,"{'quick': 2.3, 'current': 3}","{'Year': 0.4242, 'HalfYear': 0.1439, 'Month': ..."
1,52853800bb1177ca391c1800,"Alcoa, Inc.",9.02,2013-10-08 20:30:00+00:00,"{'Country': 'USA', 'Sector': 'Basic Materials'...",-0.0192,0.0823,"[0.1579, -0.0925]","[0.1899, -0.0925]",3.1,0.30,26728.11,2.02,0.0033,0.25,0.007,"{'quick': 0.7000000000000001, 'current': 1.2}","{'Year': 0.09630000000000001, 'HalfYear': 0.06..."


In [26]:
df_clean.to_csv("stocks_clean.csv", index=False)

In [27]:
df_clean.to_json("stocks_clean.json", orient='records')

In [24]:
data_cols = df_clean.columns[1:]
with open('insert_json.es', 'w') as f:
    for ind, row in df_clean.iterrows():
        ind += 1
        f.write('{"index":{"_index":"stocks","_id": ' + str(ind) + '}}' + '\n')
        
        f.write('{"fields":' 
            + str(row[data_cols].to_json(orient='index')).replace('\n', ' ') 
            + ',"id": "' + str(row.id) + '", "type": "add"}' + '\n')

    f.close()
        
    

In [66]:
import json
import re
 
# Opening JSON file
f = open('stocks_clean.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)
 
ratio_ = ["current", "quick"]
perf_ = ["Year", "Half_Year", "Month", "Week"]
desc_ = ["Country", "Sector", "Industry"]
 
# Iterating through the json
# list
# BIG O = O(n*m), n is rows and m is number of key-value pairs
with open('insert_json.cql', 'w') as f:
    for row in data:
        string = str(row).replace("'", '"')
        string = string.replace("Half Year", "Half_Year")
        string = string.replace("None", 'null')
        # Regex find empty values in dictonary
        
        # Replacing empty values with null
        pattern = r'"(\w+)": null'
        null_value_kwgargs = re.findall(pattern, string)
        for match in null_value_kwgargs:
            if match in ratio_:
                string = string.replace(f'"{match}": null', f'"{match}": 0.0')
            elif match in perf_:
                string = string.replace(f'"{match}": null', f'"{match}": 0.0')
            elif match in desc_:
                string = string.replace(f'"{match}": null', f'"{match}": ""')
            elif match == "price":
                string = string.replace(f'"{match}": null', f'"{match}": 0.0')
            else:
                continue
        
        """if len(null_value_kwgargs) > 0:
            print(null_value_kwgargs)
            print(string)
            break"""
        
        f.write(f"INSERT INTO stocks.stocks JSON '{string}';\n")
# Closing file
f.close()